In [178]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)


from datetime import date

today = str(date.today())
end_date = today

import glob

path = r'data/output/'  # use your path
allFiles = glob.glob(path + "/2*.csv")

dftrain = pd.DataFrame()

for _file in allFiles:
        try:
            dftrain = pd.concat([dftrain,pd.read_csv(_file,header=0)])
        except:
            pass

dvp = pd.read_csv('data/Defense/'+today+'.csv')
dvp = dvp.set_index("team")
dvp.sort_values(by="all",inplace=True)


for idx,row in dftrain.iterrows():
    try:
        if "opponent" in row and isinstance(row["opponent"], str):
            positions = row["Position"].lower().split("/")
            _dvp = dvp.loc[:, positions].rank().loc[row["opponent"].upper(), positions].mean()
        elif "GameInfo" in row:
            positions = row["Position"].lower().split("/")
            atHome = row["atHome"]
            game = row["GameInfo"].split(" ")[0]

            home, away = game.upper().split("@")

            opponent = away if atHome else home

            _dvp = dvp.loc[:, positions].rank().loc[opponent, positions].mean()

        dftrain.loc[idx, "dvp"] = _dvp

    except Exception as e:
        print(e)
        print("Somethng broke")
        pass

#dftrain = pd.read_csv('data/extras/train.csv', header=0)
dftest = pd.read_csv('data/output/3017-01-24.csv', header=0)
dfPredict1 = pd.read_csv('data/output/3017-01-24.csv', header=0)

In [180]:
df_train = dftrain

scaler = StandardScaler()
features = ['Salary','AvgPointsPerGame','atHome','O/U','odds',"dvp"]

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train["Final"].values
#y_train_onehot = pd.get_dummies(df_train['4GameAvg']).values

In [131]:
#df_test = dftest

#X_test = scaler.transform(df_test[features].values)
#y_test = df_test["Final"].values

In [132]:
y_train

array([ 36.  ,  37.5 ,   3.5 , ...,  12.  ,   1.25,   1.25])

In [181]:
from sklearn.ensemble import RandomForestRegressor

modelRFR = RandomForestRegressor(n_estimators=92,
                              random_state=1,
                              verbose=False,
                              warm_start=True)
modelRFR = modelRFR.fit(X_train, y_train)

dfPredict = scaler.transform(dfPredict1[features].values)
#X_test = scaler.transform(df_test[features].values)
y_tree_prediction = modelRFR.predict(dfPredict)
#print ("\naccuracy", np.sum(abs(y_tree_prediction-y_test)<5) / float(len(y_test)))

In [183]:
#y_tree_prediction
#pd.concat([pd.DataFrame(y_tree_prediction),pd.DataFrame(y_test)],axis=1)

In [185]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(12, input_dim=6, init='normal', activation='relu'))
model.add(Dense(1, init='normal', activation='relu'))
          
          
model.compile(optimizer='adam', metrics=['accuracy'],loss="mean_squared_error")

model.fit(X_train, y_train, 
          verbose=False,
          nb_epoch=150, batch_size=10)

print ('\ntime taken %s seconds' % str(time() - start))


time taken 90.26681399345398 seconds


In [38]:
#scores = model.evaluate(X_test,y_test)
#print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [186]:
#dfpredict = pd.read_csv('data/output/3017-01-21.csv', header=0)

X_test = scaler.transform(dfPredict1[features].values)

y_predict = model.predict(X_test)
y_tree_predict = modelRFR.predict(X_test)

In [197]:
output.columns

Index(['Position', 'Name', 'Salary', 'GameInfo', 'AvgPointsPerGame',
       'teamAbbrev', 'atHome', 'injured', '7GameAvg', 'FloorAvg', '4GameAvg',
       'communityBonus', 'penalty', 'dvp', 'value', 'O/U', 'odds', 'NN', 'RF'],
      dtype='object')

In [198]:
output = pd.concat([dfPredict1,
            pd.DataFrame(y_predict,columns=["NN"]),
             pd.DataFrame(y_tree_predict,columns=["RF"])]
          ,axis=1)
output.to_csv('data/output/2017-01-24.csv', sep=',', encoding='utf-8', index=False, float_format='%.3f')

In [187]:
output.loc[:,("Name","NN","RF","AvgPointsPerGame","Final","O/U","odds","dvp")].sort_values(by=["O/U","NN","dvp"], ascending=False)

,Name,NN,RF,AvgPointsPerGame,Final,O/U,odds,dvp
1,John Wall,46.020012,56.557065,48.661,NaN,217.0,1.0,22.00
2,Isaiah Thomas,42.916031,33.532609,43.647,NaN,217.0,-1.0,3.00
21,Bradley Beal,31.798546,34.847826,33.594,NaN,217.0,1.0,13.00
26,Markieff Morris,30.013466,33.176630,27.779,NaN,217.0,1.0,20.00
19,Al Horford,29.954597,35.880435,36.235,NaN,217.0,-1.0,8.00
27,Otto Porter,29.422632,34.744565,30.142,NaN,217.0,1.0,20.00
36,Marcin Gortat,27.305733,27.559783,30.892,NaN,217.0,1.0,21.00
29,Avery Bradley,25.828247,8.391304,33.458,NaN,217.0,-1.0,10.00
35,Marcus Smart,24.883583,33.660326,24.875,NaN,217.0,-1.0,6.50
40,Jae Crowder,24.077988,27.570652,26.257,NaN,217.0,-1.0,17.00


In [161]:
dfPredict1["NN2"]

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
10     0.0
11     0.0
12     0.0
13     0.0
14     0.0
15     0.0
16     0.0
17     0.0
18     0.0
19     0.0
20     0.0
21     0.0
22     0.0
23     0.0
24     0.0
25     0.0
26     0.0
27     0.0
28     0.0
29     0.0
      ... 
130    0.0
131    0.0
132    0.0
133    0.0
134    0.0
135    0.0
136    0.0
137    0.0
138    0.0
139    0.0
140    0.0
141    0.0
142    0.0
143    0.0
144    0.0
145    0.0
146    0.0
147    0.0
148    0.0
149    0.0
150    0.0
151    0.0
152    0.0
153    0.0
154    0.0
155    0.0
156    0.0
157    0.0
158    0.0
159    0.0
Name: NN2, dtype: float64